In [1]:
# %cd /home/baldeeb/Documents/Projects/ContrastiveLearningOfDenseDescriptor
%cd /home/baldeeb/Documents/Projects/dense_descriptors_and_acf/ContrastiveLearningOfDenseDescriptor

%load_ext autoreload
%autoreload 2

/home/baldeeb/Documents/Projects/dense_descriptors_and_acf/ContrastiveLearningOfDenseDescriptor


In [2]:
#temp config 
class cfg():
    dataset = 'unreal_parts'
    data_dir = '../simple_data'
    image_type = 'RGB'
    obj_class = 'mug'
    n_pair = 0
    n_nonpair_singleobj = 0 
    n_nonpair_bg = 0
    batch_size = 1
    workers = 1

    device = 'cuda:0'
    # device = 'cpu'

In [3]:
import torch 
from torchvision.models.segmentation import deeplabv3_resnet50

## Load model
file = 'temp/results_old/first_successful_run/model_checkpoints/13_09_2021__11_29_03_19_final'
device = torch.device('cuda:0')
state_dict = torch.load(file, map_location=device)
backbone = deeplabv3_resnet50(num_classes=3).to(device)
backbone.load_state_dict(state_dict['model_state_dict'])

## Load data
from dataset import make_data_loader
dataloader = make_data_loader(split='train', args=cfg())

## Run model
descriptors, images = None, None
for im, batch in dataloader:
    images = im.to(device)
    metas = batch[1]
    descriptors = backbone(images)['out']
    break


/home/baldeeb/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [4]:
# Get some resized images

from torchvision.transforms import Resize as resize

(H, W) = images[0].shape[-2:]

resized_images = []
image_scales = [1, 0.5, 0.25]

for scale in image_scales:
    resized_images.append(resize([int(H*scale), int(W*scale)])(im.clone()))

for im in resized_images:
    print(im.shape)

torch.Size([2, 3, 480, 640])
torch.Size([2, 3, 240, 320])
torch.Size([2, 3, 120, 160])


In [19]:
from loss import pyramidal_contrastive_augmentation_loss as p_loss

p_loss(resized_images, metas)

{'match': [tensor(198.9897), tensor(98.7490), tensor(49.4792)],
 'non_match': [tensor(0.5637), tensor(0.5657), tensor(0.5478)]}